In [38]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from sklearn import datasets
import numpy as np
from sklearn.model_selection import train_test_split
data = datasets.load_breast_cancer()
x,y = data.data, data.target
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

n_samples, n_feature = x.shape
x_train = torch.from_numpy(x_train.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_train = y_train.view(-1,1)

x_test = torch.from_numpy(x_test.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))
y_test = y_test.view(-1,1)

In [93]:
torch.cuda.is_available()

False

In [40]:
class LogisticRegression(nn.Module):
    def __init__(self,n_input_feature):
        super(LogisticRegression,self).__init__()
        self.linear = nn.Linear(n_input_feature,1)
    def forward(self,x):
        z = torch.sigmoid(self.linear(x))
        return z
model = LogisticRegression(n_feature)

In [83]:
max_epochs = 10000
loss = nn.BCELoss()
rate = 1e-2
optimizer = torch.optim.SGD(model.parameters(),lr = rate)

for epoch in range(max_epochs):
    z = model(x_train)
    J = loss(z,y_train)
    J.backward()
    optimizer.step()
    optimizer.zero_grad()
    if epoch % 1000 == 0:
        print(f'epoch {epoch}: loss = {J.item():.5f}')

epoch 0: loss = 0.05353
epoch 1000: loss = 0.05318
epoch 2000: loss = 0.05283
epoch 3000: loss = 0.05250
epoch 4000: loss = 0.05218
epoch 5000: loss = 0.05188
epoch 6000: loss = 0.05158
epoch 7000: loss = 0.05130
epoch 8000: loss = 0.05102
epoch 9000: loss = 0.05075


In [84]:
with torch.no_grad():
    z_x = model(x_train)
    z_x0 = model(x_test)
    z_x = z_x.round()
    z_x0 = z_x0.round()
    acc = z_x.eq(y_train).sum()/y_train.shape[0]
    acc0 = z_x0.eq(y_test).sum()/y_test.shape[0]
print(f'Training data accuracy = {acc*100:.5f}%')
print(f'Test data accuracy = {acc0*100:.5f}%')

Training data accuracy = 98.68132%
Test data accuracy = 98.24561%


In [3]:
import torch
print(torch.finfo().max)